# Collection of implemented Greedy Algorithms for the lecture Algorithm techniques SS25 HTWG Konstanz

# A* Algorithm

The A* algorithm uses a heuristik to determine the best way to move through a graph from a starting point to an endpoint.

In [ ]:
heuristic = {
    "A": 6, 
    "B": 4,
    "C": 6,
    "D": 5,
    "E": 2,
    "F": 0
}


graph = {
    "A": [("B", 2), ("C", 4)],
    "B": [("D", 3), ("E", 2)],
    "C": [("D", 1)],
    "D": [("F", 5)],
    "E": [("F", 2)],
    "F": []
}

def pop_best(list):
   
  best = sorted(list, key=lambda x: x[1])[0]
  list.remove(best)

  return best
      

def a_star(priority_list=[('A', 0)], visited=[], curr_cost=0,start_node='A', end_node='F', optimal=[]):


    while (priority_list):

      best = pop_best(priority_list)
      visited.append(best[0])
      optimal.append(best)

      if best[0] == end_node: return optimal

      current = 0 if best[0] == start_node else best[1] - heuristic[best[0]]
      for child in graph[best[0]]:
        if child[0] in visited:
           continue

        way_cost = current + child[1]
        estimated = way_cost + heuristic[child[0]]
        priority_list.append((child[0], estimated))

best_path = a_star()
print(best_path)


[('A', 0), ('B', 6), ('E', 6), ('F', 6)]


# Greedy optimal activity selection algorithm

In [55]:
activities = [
    ("A1", 0, 6),
    ("A2", 1, 4),
    ("A3", 3, 5),
    ("A4", 5, 7),
    ("A5", 8, 9),
    ("A6", 5, 9),
    ("A7", 2, 13),
    ("A8", 6, 10),
    ("A9", 8, 11),
    ("A10", 12, 14),
    ("A11", 13, 16),
    ("A12", 0, 3),
    ("A13", 4, 5),
    ("A14", 9, 10),
    ("A15", 11, 12),
]



def optimal_plan(activities=activities):
  activities = sorted(activities, key=lambda x: x[2])
  best_activities = []
  curr_time = 0
  while list(filter(lambda x: x[1] >= curr_time, activities)):
    filtered = list(filter(lambda x: x[1] >= curr_time, activities))
    best = filtered.pop(0)
    best_activities.append(best)
    curr_time = best[2]
    
  return best_activities

res = optimal_plan()
print(res)

[('A12', 0, 3), ('A3', 3, 5), ('A4', 5, 7), ('A5', 8, 9), ('A14', 9, 10), ('A15', 11, 12), ('A10', 12, 14)]


# Approx Bin Packing

Ziel ist es eine Menge an Items mit unterschiedlicher groesse in moeglichst wenige pakete zu packen die eine fixe groese haben.

In [66]:
packets = [2, 5, 4, 7, 1, 3, 8]
bin_capacity = 10

def fits(packet, bins):
  for bin in bins:
    if sum(bin) + packet <= bin_capacity:
      return bin
    
  return None

def bin_packing(packets=packets):
  
  curr_bins = []
  packets = sorted(packets, reverse=True) # to sort the array descending the approximation get closer to optimal :)
  
  while(packets):
    packet = packets.pop(0)
    
    if not curr_bins:
      curr_bins.append([packet])
    else:
      bin = fits(packet, curr_bins)
      bin.append(packet) if bin else curr_bins.append([packet])
  return curr_bins


res = bin_packing()
print(res) 

[[8, 2], [7, 3], [5, 4, 1]]


# Kruskal's Greedy Based MST algorithm

Ziel ist es einen Minimal spannenden Baum ueber alle Knoten eines Graphen zu bilden.


In [120]:
edges = [
    ("A", "B", 1),
    ("A", "C", 3),
    ("A", "D", 4),
    ("B", "C", 2),
    ("C", "D", 5),
]

edges2 = [
    ("A", "B", 4),
    ("A", "H", 8),
    ("B", "C", 8),
    ("B", "H", 11),
    ("C", "D", 7),
    ("C", "F", 4),
    ("C", "I", 2),
    ("D", "E", 9),
    ("D", "F", 14),
    ("E", "F", 10),
    ("F", "G", 2),
    ("G", "H", 1),
    ("G", "I", 6),
    ("H", "I", 7),
    ("B", "E", 6),
]

def merge_trees(edge, sub_trees):
  t1 = next(filter(lambda x: edge[0] in x,sub_trees))
  t2 = next(filter(lambda x: edge[1] in x,sub_trees))

  sub_trees.remove(t1)
  sub_trees.remove(t2)
  sub_trees.append(t1 + t2)

def is_cycle(edge, sub_trees):

  for tree in sub_trees:
    if edge[0] in tree and edge[1] in tree:
      return True
  return False


def mst(edges=edges, min_tree=[]):

  sub_trees = [[node] for node in sorted({n for edge in edges for n in edge[:2]})]


  edges = sorted(edges, key=lambda x: x[2])
  while edges:
    min_edge = edges.pop(0)
    if is_cycle(min_edge, sub_trees):
      continue
    min_tree.append(min_edge)
    merge_trees(min_edge, sub_trees)

  return min_tree

res = mst(edges=edges2, min_tree=[])
print(res)

cost = sum(w for _, _, w in res)
print(cost)

[('G', 'H', 1), ('C', 'I', 2), ('F', 'G', 2), ('A', 'B', 4), ('C', 'F', 4), ('B', 'E', 6), ('C', 'D', 7), ('A', 'H', 8)]
34


# Graph Splitting in k Clusters using MST

Ziel ist es einen Graphen in k Cluster zu unterteilen mit minimaler Spannlaenge.

In [125]:
import copy

edges = [
    ("A", "B", 1),
    ("A", "C", 3),
    ("A", "D", 4),
    ("B", "C", 2),
    ("C", "D", 5),
]

edges2 = [
    ("A", "B", 4),
    ("A", "H", 8),
    ("B", "C", 8),
    ("B", "H", 11),
    ("C", "D", 7),
    ("C", "F", 4),
    ("C", "I", 2),
    ("D", "E", 9),
    ("D", "F", 14),
    ("E", "F", 10),
    ("F", "G", 2),
    ("G", "H", 1),
    ("G", "I", 6),
    ("H", "I", 7),
    ("B", "E", 6),
]

sub_trees = [[node] for node in sorted({n for edge in edges2 for n in edge[:2]})]

def build_clusters(edges, sub_trees):
  for edge in edges:
    t1 = next(filter(lambda x: edge[0] in x,sub_trees))
    t2 = next(filter(lambda x: edge[1] in x,sub_trees))

    sub_trees.remove(t1)
    sub_trees.remove(t2)
    sub_trees.append(t1 + t2)

  return sub_trees

def k_clustering(k, edges):
  sub_trees = [[node] for node in sorted({n for edge in edges for n in edge[:2]})]

  minimal_tree = mst(edges=edges, min_tree=[])
  descending_sorted = sorted(minimal_tree, key=lambda x: x[2], reverse=True)
  return build_clusters(descending_sorted[k - 1:], sub_trees)

res = k_clustering(3, edges2)
print(res)


[['D'], ['A', 'B', 'E'], ['C', 'F', 'I', 'G', 'H']]


# Djikstra Algorithm Greedy Approach

Ziel ist es alle kuerzesten Pfade eines gewaehlten Knotens zu allen anderen Knoten zu finden.

In [18]:
import math

graph = {
    0: [(1, 2), (2, 5)],
    1: [(0, 2), (3, 4), (4, 7)],
    2: [(0, 5), (5, 3)],
    3: [(1, 4), (4, 1)],
    4: [(1, 7), (3, 1), (5, 2)],
    5: [(2, 3), (4, 2)]
}

def dijkstra(graph):
  v = [x for x in graph.keys()]
  distance = [None for _ in graph.keys()]
  pred = [None for _ in graph.keys()]
  candidates = [(0, 0)]
  visited = []

  while candidates:
    best = candidates.pop(0)
    visited.append(best[0])

    for child in graph[best[0]]:
        if child[0] not in visited:
            new_dist = best[1] + child[1]
            if not distance[child[0]] or new_dist < distance[child[0]]:
                distance[child[0]] = new_dist
                pred[child[0]] = best[0]
                candidates.append((child[0], new_dist))
              
    candidates = sorted(candidates, key=lambda x: distance[x[0]])

  return v, distance, pred

v, distance, pred = dijkstra(graph)
print(v, distance, pred)

[0, 1, 2, 3, 4, 5] [None, 2, 5, 6, 7, 8] [None, 0, 0, 1, 3, 2]


# Greedy approach for coloring a graph

Gegeben sein eine Anzahl n von Ländern und eine dazugehörige Landkarte als Matrix
a[][], in der a[i][j]=1 ist, wenn die Länder i und j benachbart sind, und
a[i][j]=0 sonst.

In [76]:
m = [[0,1,1,0,0],
     [1,0,1,1,0],  
     [1,1,0,1,1],  
     [0,1,1,0,1],  
     [0,0,1,1,0]]

def sort_by_edges(graph):
  return sorted([(sum(row), index) for index, row in enumerate(graph)], key=lambda x: x[0], reverse=True)

def get_optimal_color(colors, node, graph):
  index = node[1]
  adj_colors = []
  optimal_color = 0

  for index, j in enumerate(graph[index]):
    if j:
      adj_colors.append(colors[index])
  
  while(optimal_color in adj_colors): optimal_color+=1
  return optimal_color

def color_graph(graph):

  colors = [-1 for _ in range(len(m))]
  nodes = sort_by_edges(graph)
 
  while nodes:
    n = nodes.pop(0)

    c = get_optimal_color(colors, n, graph)
    colors[n[1]] = c
  
  return colors

    

  


res = color_graph(m)
print(res)

[2, 1, 0, 2, 1]


# Job Scheduling on n Processors with max(deadline) slots per processor.

Given a list of jobs (with profits and deadlines), and multiple processors, find a scheduling of as many jobs as possible within their deadlines to maximize total profit.

In [102]:
jobs = [
    (100, 2),
    (19, 1),
    (27, 2),
    (25, 1),
    (15, 3),
    (30, 3),
    (50, 2)
]

num_processors = 3

def set_to_processor(job, processors):
  for i in range(job[1], 0, -1):
    
    for processor in processors:
      if not processor[i - 1]:
        processor[i - 1].append(job)
        
        return

def job_scheduling(num_processors, jobs):
  jobs = sorted(jobs, key=lambda x: x[0], reverse=True)
  max_deadline = max(jobs, key=lambda x: x[1])[1]

  slots_per_processor = [[[] for _ in range(max_deadline)] for _ in range(num_processors)] 

  while jobs:
    best = jobs.pop(0)
    set_to_processor(best, slots_per_processor)

  sums = 0
  for processor in slots_per_processor:
    for slot in processor:
      if slot:
        sums += slot[0][0]

  return sum(sum(entry[0][0] if entry else 0 for entry in row) for row in slots_per_processor)

res = job_scheduling(num_processors, jobs)
print(res)

266


# Fraktionales Rucksack Problem

Aehnlich wie das normale Rucksack Problem bei dem man Gegenstaende die ein Gewicht und ein Wert haben so in einen Rucksack mit begrenzter kapazitaet zu packen das 
man den summierten Wert maximiert.

Beim Fraktionalen Problem kommt noch die Bedingung dazu dass man auch nur Bruchteile von Gegenstaenden mit nehmen kann.

In [114]:
# value, weight
items = [
    (60, 10),
    (100, 20),
    (120, 30)
]

capacity = 50



def fractional_knapsack(items, capacity):

  items = sorted([(item[0] / item[1], item[0], item[1]) for item in items], key=lambda x: x[0], reverse=True)

  values = []

  while(items):

    best = items.pop(0)

    if capacity - best[2] >= 0:
      capacity -= best[2]
      values.append(best[1])

    else:
      fraction = capacity * best[1] / best[2]
      capacity -= fraction
      values.append(fraction)
      return sum(values)
    
  return sum(values)



res = fractional_knapsack(items, capacity)

print(res)


240.0


# Maximum-Weight Independent Set (MWIS)

gesucht ist das independent set mit der approximativ hoechsten Summe durch einen Greedy Approach.

{1, 3, 5, 2, 8} = 1 + 5 + 8 = 14

In [ ]:
weights = [4, 5, 4, 1, 7]

def mwis(weights):
    weights = weights[:]
    selected = []

    i = 0
    while i < len(weights):
        max_weight = max(weights)
        max_index = weights.index(max_weight)

        selected.append(max_weight)

        del weights[max_index]
        if max_index - 1 >= 0:
            del weights[max_index - 1]
            max_index -= 1
        if max_index < len(weights):
            del weights[max_index]

    return selected, sum(selected)

weights = [4, 5, 4, 1, 7]
v, s = mwis(weights)
print(v, s)
    

  
v, s = mwis(weights)
print(v, s)


[4, 5, 4, 1, 7]
[4, 5, 4]
[4]
[7, 4, 4] 15


# Bellman-Ford Algorithm

Algorithmus um alle kürzesten Wege in einem Unidirektionalen Graphen mit negativen Kantengewichten von einem Startpunkt s zu allen anderen Knoten zu finden

In [17]:
import math

graph = {
    0: [(1, 4), (2, 2)],
    1: [(2, 3), (3, 2), (4, 3)],
    2: [(1, 1), (3, 4), (4, 5)],
    3: [],
    4: [(3, -5)]
}





def bellman_ford(graph):
  v = [0, math.inf, math.inf, math.inf, math.inf]
  changes = True

  while(changes):
    changes = False

    for node in graph:
      for child in graph[node]:

        id = child[0]
        cost = child[1]

        if v[id] == math.inf or v[id] > (v[node] + cost):
          changes = True
          v[id] = cost + v[node]
  return v
        


res = bellman_ford(graph)
print(res)




[0, 3, 2, 1, 6]


# Greedy Approach to the Partitioning Problem

Das Partitiosproblem returned im Greedy Approach eine approximativ korrekte Lösung die nicht immer komplett korrekt ist.

In [20]:
n = [9, 7, 5, 3, 1]



def partition(n):

  a1, a2 = [], []
  s1, s2 = 0, 0
  n = sorted(n, reverse=True)


  while(n):
    m = n.pop(0)

    if s1 > s2:
      a2.append(m)
      s2 += m
    else:
      a1.append(m)
      s1 += m

  return a1, a2


a1, a2 = partition(n)
print(sum(a1))
print(sum(a2))

13
12


# Set Cover Problem

Gegeben sei eine Menge u mit n Elementen und einige Teilmengen welche alle Teilmengen von U sind.

Ziel ist es die niedrigste Anzahl an Teilmengen zu finden die insgesamt die Gesamtmenge U abbilden.

Greedy ist nicht optimal obviously weil er nicht den ganzen Baum abläuft sondern immer nur von den noch übrigen Teilmengen diese nimmt die am meisten noch nicht vorhandene Elemente besitzt.

In [26]:
u = {1, 2, 3, 4, 5, 6, 7}

subsets = [
    {1, 2, 3},
    {2, 4},   
    {3, 4, 5},
    {4, 5, 6},
    {5, 6, 7},
    {1, 7}    
]

def get_best(subsets, curr_cover):
  best = None
  best_counter = 0

  for set in subsets:
    counter = 0
    for item in set:
      if item not in curr_cover:
        counter+=1
    if counter > best_counter:
      best_counter = counter
      best = set
  return best


def set_cover(u, subsets):
  used_sets = []
  curr_cover = set()

  while curr_cover != u:

    best = get_best(subsets, curr_cover)
    used_sets.append(best)
    curr_cover.update(best)

  return used_sets

res = set_cover(u, subsets)
print(res)



[{1, 2, 3}, {4, 5, 6}, {5, 6, 7}]


# Shanon Code (chatGPT implementierung weil kb auf den mist)

In [29]:
import math

def to_binary(fraction, length):
    binary = ""
    while len(binary) < length:
        fraction *= 2
        bit = int(fraction)
        binary += str(bit)
        fraction -= bit
    return binary

def shannon_code(symbols, probabilities):
    paired = sorted(zip(symbols, probabilities), key=lambda x: -x[1])
    
    codes = {}
    cumulative_probability = 0

    for symbol, p in paired:
        l = math.ceil(-math.log2(p))
        f = cumulative_probability  
        code = to_binary(f, l)      
        codes[symbol] = code
        cumulative_probability += p

    return codes
  

symbols = ['A', 'B', 'C', 'D']
probabilities = [0.4, 0.3, 0.2, 0.1]

codes = shannon_code(symbols, probabilities)
print(codes)

{'A': '00', 'B': '01', 'C': '101', 'D': '1110'}


# Shortest common superstring

Gegebenen sei eine Menge an Strings die man durch eventuelles überlappen zu einem Gesamtstring mergen soll der eine möglichst geringe Länge hat.

e.g. [abc, cde, efg] = abcdefg -> len(7)
